In [ ]:
%matplotlib inline

# Parameter space.

In this example,
we will see the basics of [ParameterSpace][gemseo.algos.parameter_space.ParameterSpace].


In [ ]:
from __future__ import annotations

from gemseo import create_discipline
from gemseo import sample_disciplines
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.post.dataset.scatter_plot_matrix import ScatterMatrix
from gemseo.settings.probability_distributions import SPNormalDistribution_Settings

## Create a parameter space

Firstly,
the creation of a [ParameterSpace][gemseo.algos.parameter_space.ParameterSpace] does not require any mandatory argument:



In [ ]:
parameter_space = ParameterSpace()

Then, we can add either deterministic variables
from their lower and upper bounds
(use [add_variable()][gemseo.algos.parameter_space.ParameterSpace.add_variable]):



In [ ]:
parameter_space.add_variable("x", lower_bound=-2.0, upper_bound=2.0)

or uncertain variables from their distribution names and parameters
(use [add_random_variable()][gemseo.algos.parameter_space.ParameterSpace.add_random_variable]):



In [ ]:
parameter_space.add_random_variable(
    "y", SPNormalDistribution_Settings(mu=0.0, sigma=1.0)
)
parameter_space

or without Pydantic model:

``` python
parameter_space.add_random_variable("y", "SPNormalDistribution", mu=0.0, sigma=1.0)
```

!!! warning

    We cannot mix probability distributions from different families,
    e.g. an [OTDistribution][gemseo.uncertainty.distributions.openturns.distribution.OTDistribution] and a [SPDistribution][gemseo.uncertainty.distributions.scipy.distribution.SPDistribution].

!!! note

    [This page][available-probability-distributions]
    lists the available probability distributions.
    The names prefixed with SP correspond to distributions based on SciPy.
    while those prefixed with OT correspond to distributions based on OpenTURNS.
    The settings class `DistributionClassName_Settings`
    associated with the distribution `DistributionClassName` can be imported
    from
    [gemseo.settings.probability_distributions][gemseo.settings.probability_distributions].

We can check that the variables *x* and *y* are implemented
as deterministic and uncertain variables respectively:



In [ ]:
parameter_space.is_deterministic("x"), parameter_space.is_uncertain("y")

Note that when GEMSEO does not offer a class for the SciPy distribution,
we can use the generic GEMSEO class [SPDistribution][gemseo.uncertainty.distributions.scipy.distribution.SPDistribution]
to create any SciPy distribution
by setting `interfaced_distribution` to its SciPy name
and `parameters` as a dictionary of SciPy parameter names and values
([see the documentation of SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)).

``` python
parameter_space.add_random_variable(
    "y",
    SPDistribution_Settings(interface_distribution="norm", parameters={"loc": 1.0, "scale": 2.0}),
)
```

A similar procedure can be followed
for OpenTURNS distributions for which
GEMSEO does not offer a class directly.
We can use the generic GEMSEO class [OTDistribution][gemseo.uncertainty.distributions.openturns.distribution.OTDistribution]
to create any OpenTURNS distribution
by setting `interfaced_distribution` to its OpenTURNS name
and `parameters` as a tuple of OpenTURNS parameter values
([see the documentation of OpenTURNS](https://openturns.github.io/openturns/latest/user_manual/_generated/openturns.Normal.html#openturns.Normal)).

``` python
parameter_space.add_random_variable(
    "y",
    OTDistribution_Settings(interface_distribution="Normal", parameters=(1.0, 2.0)),
)
```

## Sample from the parameter space

We can sample the uncertain variables from the [ParameterSpace][gemseo.algos.parameter_space.ParameterSpace]
and return the result either as a dictionary of NumPy arrays
indexed by the names of these variables:



In [ ]:
samples = parameter_space.compute_samples(n_samples=4)
samples

or a unique NumPy array concatenating the values of these variables
according to the order in which they were added to the [ParameterSpace][gemseo.algos.parameter_space.ParameterSpace]:



In [ ]:
samples = parameter_space.compute_samples(n_samples=2, as_dict=True)
samples

## Sample a discipline over the parameter space

We can also sample a discipline over the parameter space.
For simplicity,
we instantiate an [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline] from a dictionary of expressions:



In [ ]:
discipline = create_discipline("AnalyticDiscipline", expressions={"z": "x+y"})

Then,
we use the [sample_disciplines()][gemseo.sample_disciplines] function
with an LHS algorithm
to generate 100 samples of the discipline
over the whole parameter space:



In [ ]:
dataset = sample_disciplines(
    [discipline], parameter_space, "z", algo_name="PYDOE_LHS", n_samples=100
)

and visualize it in a tabular way:



In [ ]:
dataset

or with a graphical post-processing,
e.g. a scatter plot matrix:



In [ ]:
ScatterMatrix(dataset).execute(save=False, show=True)

## Sample a discipline over the uncertain space

If we want to sample a discipline over the uncertain space,
we need to filter the uncertain variables:



In [ ]:
parameter_space.filter(parameter_space.uncertain_variables)

If we want to sample a discipline over the uncertain space only,
we need to extract it:



In [ ]:
uncertain_space = parameter_space.extract_uncertain_space()

Then, we sample the discipline over this uncertain space:



In [ ]:
dataset = sample_disciplines(
    [discipline], uncertain_space, "z", algo_name="PYDOE_LHS", n_samples=100
)
dataset